# LDA using Spacey and Gensim
## with n-grams for more accurate topics

- [ ] set working directory if needed
- [ ] still lots of human level interpretation needed - there are lots of scientific writing terms still
- [ ] explain how to set up anaconda environment OR
- [ ] export, create new venv, and import dedicated env for this notebook
- [ ] then, set working directory to desired folder


### Source:
Anika Nissen (University of Duisburg-Essen (UDE)) and Dominic Rosati (scite.ai)

In [2]:
import os
path = '/Users/nicolapoppy/Desktop/OCR'
os.chdir(path)
os.getcwd()

'/Users/nicolapoppy/Desktop/OCR'

## dependencies

In [3]:
# Install
!pip install pyLDAvis -qq 
!pip install -qq -U gensim
!pip install spacy -qq
!pip install matplotlib -qq
!pip install seaborn -qq
!python -m spacy download en_core_web_md -qq
!pip install fsspec

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
top2vec 1.0.26 requires gensim<4.0.0, but you have gensim 4.2.0 which is incompatible.
^C
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/runpy.py", line 185, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/opt/anaconda3/lib/python3.8/runpy.py", line 144, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/opt/anaconda3/lib/python3.8/runpy.py", line 111, in _get_module_details
    __import__(pkg_name)
  File "/opt/anaconda3/lib/python3.8/site-packages/spacy/__init__.py", line 11, in <module>
    from thinc.api import prefer_gpu, require_gpu, require_cpu  # noqa: F401
  File "/opt/anaconda3/lib/python3.8/site-packages/thinc/__init__.py", line 5, in <module>
    from .config import registry
  File "/opt/anac

In [4]:
import warnings
warnings.filterwarnings('ignore')
# Import
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spacy
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()# Visualise inside a notebook
import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
import itertools

pd.set_option('display.max_columns', None)

/opt/anaconda3/envs/LDA_Spacey_Gensim/lib/python3.8/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


In [32]:
# Read the data
path = '/Users/nicolapoppy/Desktop/CODING_WORKING/Python/LIBRARIES/LDA_with_ngrams_Spacy/TE SLR Corpus.csv'
reports = pd.read_csv(path)
reports.head()
reports.info()

# Our spaCy model:
nlp = en_core_web_md.load()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 87 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Key                   118 non-null    object 
 1   Item Type             118 non-null    object 
 2   Publication Year      117 non-null    float64
 3   Author                118 non-null    object 
 4   Title                 118 non-null    object 
 5   Publication Title     115 non-null    object 
 6   ISBN                  0 non-null      float64
 7   ISSN                  113 non-null    object 
 8   DOI                   110 non-null    object 
 9   Url                   112 non-null    object 
 10  summary               113 non-null    object 
 11  Date                  117 non-null    object 
 12  Date Added            118 non-null    object 
 13  Date Modified         118 non-null    object 
 14  Access Date           112 non-null    object 
 15  Pages                 1

In [38]:
reports['summary'] = reports['summary'].astype(str)
reports.head()
print(reports.dtypes)

Key                  object
Item Type            object
Publication Year    float64
Author               object
Title                object
                     ...   
Section             float64
Session             float64
Committee           float64
History             float64
Legislative Body    float64
Length: 87, dtype: object


## set the number of n_grams here

In [39]:
from nltk import ngrams

def compile_ngrams(text, number_of_n=3, include_unigrams=False):
  ngram_list = []
  # number_of_n controls up to how many n we build an ngram for
  # 2 being bigrams, 3 being trigrams, ect.
  for n in range(number_of_n):
    if n == 0 and not include_unigrams:
      continue
    for ngram in ngrams(text.split(), n + 1):
      ngram_list.append(' '.join(ngram))
  return ngram_list

"""this is a test to make sure the function works"""
print(compile_ngrams("There was a cloud computing conference about big data and natural language processing"))

['There was', 'was a', 'a cloud', 'cloud computing', 'computing conference', 'conference about', 'about big', 'big data', 'data and', 'and natural', 'natural language', 'language processing', 'There was a', 'was a cloud', 'a cloud computing', 'cloud computing conference', 'computing conference about', 'conference about big', 'about big data', 'big data and', 'data and natural', 'and natural language', 'natural language processing']


In [40]:
# Tags I want to remove from the text
removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']
tokens = []
# words I really care about that should certainly be in the dictionary
author_assigned_keywords = ["multiple choice machine comprehension"]

for summary in nlp.pipe(reports['summary']):
   # build up tokens here
   # using the authors heuristics:
   unigrams = [token.lemma_.lower() for token in summary if token.pos_ not in removal and not token.is_stop and token.is_alpha]
   # using ngrams
   # we can use the proj_tok clean text as the "paragraph"
   proj_tok = compile_ngrams(" ".join(unigrams), number_of_n=2, include_unigrams=False)
   # using "author assigned keywords" by checking if they are in the text
   proj_tok += [keyword for keyword in author_assigned_keywords if keyword in summary.text]
   tokens.append(proj_tok)

# Add tokens to new column
reports['tokens'] = tokens
reports['tokens']

# Create dictionary
# I will apply the Dictionary Object from Gensim, which maps each word to their unique ID:
dictionary = Dictionary(reports['tokens'])
print(dictionary.token2id)

{'academic growth': 0, 'access modern': 1, 'accord property': 2, 'achievement advance': 3, 'achievement datum': 4, 'achievement evaluate': 5, 'achievement find': 6, 'achievement growth': 7, 'acquire valuable': 8, 'advance understanding': 9, 'affect teacher': 10, 'align grow': 11, 'analysis describe': 12, 'apparent relationship': 13, 'appear contribute': 14, 'article combine': 15, 'ask research': 16, 'assign public': 17, 'attend mount': 18, 'attend place': 19, 'attention define': 20, 'background contexteducational': 21, 'base condition': 22, 'begin recognize': 23, 'body work': 24, 'broad conception': 25, 'build body': 26, 'build strong': 27, 'career intention': 28, 'career plan': 29, 'challenge pose': 30, 'challenge school': 31, 'characteristic condition': 32, 'characteristic school': 33, 'characteristic wide': 34, 'choose leave': 35, 'clean maintain': 36, 'combine statewide': 37, 'common school': 38, 'compare school': 39, 'conceive working': 40, 'conception context': 41, 'conclusion re

## set number of topics here
look for num_topics = 

In [42]:

# Filter dictionary
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)

# Create corpus
corpus = [dictionary.doc2bow(doc) for doc in reports['tokens']]

# LDA model building
# lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=50, num_topics=10, workers = 4, passes=10)

# # Coherence score using C_umass:
# topics = []
# score = []
# for i in range(1,20,1):
#    lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=10, num_topics=i, workers = 4, passes=10, random_state=100)
#    cm = CoherenceModel(model=lda_model, corpus=corpus, dictionary=dictionary, coherence='u_mass')
#    topics.append(i)
#    score.append(cm.get_coherence())
# _=plt.plot(topics, score)
# _=plt.xlabel('Number of Topics')
# _=plt.ylabel('Coherence Score')
# plt.show()

# # Coherence score using C_v:
# topics = []
# score = []
# for i in range(1,20,1):
#    lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=10, num_topics=i, workers = 4, passes=10, random_state=100)
#    cm = CoherenceModel(model=lda_model, texts = reports['tokens'], corpus=corpus, dictionary=dictionary, coherence='c_v')
#    topics.append(i)
#    score.append(cm.get_coherence())
# _=plt.plot(topics, score)
# _=plt.xlabel('Number of Topics')
# _=plt.ylabel('Coherence Score')
# plt.show()

# Optimal model
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=100, num_topics=5, workers = 4, passes=100)

# Print topics
lda_model.print_topics(-1)

# Where does a text belong to
lda_model[corpus][0]
reports['summary'][0]

# Visualize topics
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_display)

# Save the report
pyLDAvis.save_html(lda_display, 'index.html')

In [43]:
lda_model.print_topics()

[(0,
  '0.202*"teaching effectiveness" + 0.175*"improve teaching" + 0.107*"student evaluation" + 0.082*"future research" + 0.082*"teach effectiveness" + 0.069*"effectiveness student" + 0.043*"effectiveness research" + 0.042*"faculty development" + 0.029*"control group" + 0.017*"teacher education"'),
 (1,
  '0.160*"self efficacy" + 0.136*"effective teaching" + 0.126*"medical education" + 0.092*"teaching learning" + 0.083*"teaching method" + 0.052*"content knowledge" + 0.047*"purpose study" + 0.047*"faculty development" + 0.036*"study examine" + 0.036*"teaching learn"'),
 (2,
  '0.176*"effective teacher" + 0.155*"student achievement" + 0.092*"teacher effectiveness" + 0.071*"quality teacher" + 0.071*"effect size" + 0.063*"teacher education" + 0.050*"teacher school" + 0.036*"teacher quality" + 0.031*"student performance" + 0.030*"teacher student"'),
 (3,
  '0.242*"student learning" + 0.200*"medical school" + 0.115*"medical student" + 0.087*"control trial" + 0.070*"teaching method" + 0.056*

In [44]:
lda_model[corpus][1]
lda_model[corpus][1][3][0]
print('cluster ' + str(lda_model[corpus][1][3][0]) + ' is the most highest value')
print()
print(lda_model[corpus][1])

cluster 3 is the most highest value

[(0, 0.04000392), (1, 0.04182488), (2, 0.041136827), (3, 0.040156133), (4, 0.8368782)]


In [45]:
reports['summary'][1]

"Effective teaching skills consist of high levels of student engagement based on good classroom and time management skills; the ability to scaffold learning that is adapted to students' current levels of understanding; cognitively engaging students in higher-order thinking; and encouraging and supporting success. The research reported here suggests that in elementary classrooms, effective teaching skills are effective for all students, both with and without special education needs. Drawing on a research programme extending over nearly two decades, we make the case that effective inclusionary practices, and therefore overall effective teaching, depend in part on the beliefs of teachers about the nature of disability, and about their roles and responsibilities in working with students with special education needs. Elementary classroom teachers who believe students with special needs are their responsibility tend to be more effective overall with all of their students. We provide evidence

/opt/anaconda3/envs/LDA_Spacey_Gensim/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/LDA_Spacey_Gensim/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/LDA_Spacey_Gensim/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/LDA_Spacey_Gensim/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/LDA_Spacey_G

## put cluster back in the df and export csv
- [ ] add cluster column
- [ ] use lda_model[corpus][i] max value?
- [ ] fill column with lda_model[corpus][i] values for more detail